In [ ]:
# Setting up the Colab environment. DO NOT EDIT!
import os
#import warnings
#warnings.filterwarnings("ignore")

try:
    import otter, pingouin

except ImportError:
    ! pip install -q otter-grader==4.0.0, pingouin
    import otter

if not os.path.exists('walkthrough-tests'):
    zip_files = [f for f in os.listdir() if f.endswith('.zip')]
    assert len(zip_files)>0, 'Could not find any zip files!'
    assert len(zip_files)==1, 'Found multiple zip files!'
    ! unzip {zip_files[0]}

grader = otter.Notebook(colab=True,
                        tests_dir = 'walkthrough-tests')

# Walkthrough

## Learning Objectives
At the end of this learning activity you will be able to:
 - Practice using `pg.normality` and `pg.qqplot` to assess normality.
 - Practice using `pg.linear_regression` to perform multiple regression.
 - Interpret the results of linear regression such as the coefficient, p-value, R^2, and confidence intervals.
 - Describe a _residual_ and how to interpret it.
 - Relate the _dummy variable trap_ and how to avoid it during regression.
 - Describe _overfitting_ and how to avoid it.

As we discussed with Dr. Devlin in the introduction video, this week and next we are going to look at HIV neurocognitive impairment data from a cohort here at Drexel.
Each person was given a full-scale neuropsychological exam and the resulting values were aggregated and normalized into Z-scores based on demographically matched healthy individuals.

In this walkthrough we will explore the effects of antiretroviral medications on neurological impairment.
In our cohort, we have two major drug regimens, d4T (Stavudine) and the newer Emtricitabine/tenofovir (Truvada).
The older Stavudine is suspected to have neurotoxic effects that are not found in the newer Truvada.
We will use inferential statistics to understand this effect.

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import pingouin as pg

%matplotlib inline

In [ ]:
data = pd.read_csv('hiv_neuro_data.csv')
data['education'] = data['education'].astype(float)
data.head()

Before we start, we need to talk about assumptions.

Basic linear regression has a number assumptions baked into itself:
 - **Linearity**: The relationship between the independent variables (predictors) and the dependent variable (outcome) is linear. This means that changes in the predictors lead to proportional changes in the dependent variable.
 - **The relationship between the independent variables and the dependent variable is additive**: The effect of changes in an independent variable X on the dependent variable Y is consistent, regardless of the values of other independent variables. This assumption might not hold if there are interaction effects between independent variables that affect the dependent variable.
 - **Independence**: Observations are independent of each other. This means that the observations do not influence each other, an assumption that is particularly important in time-series data where time-related dependencies can violate this assumption.
 - **Homoscedasticity**: The variance of error terms (residuals) is constant across all levels of the independent variables. In other words, as the predictor variable increases, the spread (variance) of the residuals remains constant. This is evaluated at the **end** of the fit.
 - **Normal Distribution of Errors**: The residuals (errors) of the model are normally distributed. This assumption is especially important for hypothesis testing (e.g., t-tests of coefficients) and confidence interval construction. It's worth noting that for large sample sizes, the Central Limit Theorem helps mitigate the violation of this assumption. This is evaluated at the **end** of the fit.
 - **Minimal Multicollinearity**: The independent variables need to be independent of each other. Multicollinearity doesn't affect the fit of the model as much as it affects the coefficients' estimates, making them unstable and difficult to interpret.
 - **No perfect multicollinearity**: Also called the _dummy variable trap_. It states that none of the independent variables should be a perfect linear function of other independent variables. We'll talk more about this when we run into it.

Biology itself is highly non-linear.
That doesn't mean we can't use linear assumptions to explore biological questions, it just means that we need to be mindful when interpretting the results.

## Exploration

Let's start by plotting the each variable against EDZ.

In [ ]:
fig, (age_ax, edu_ax, ys_ax) = plt.subplots(1,3, sharey=True, figsize = (15, 5))

sns.regplot(data = data,
            x = 'age',
            y = 'exec_domain_z',
            ax=age_ax)

sns.regplot(data = data,
            x = 'education',
            y = 'exec_domain_z',
            ax=edu_ax)

sns.regplot(data = data,
            x = 'YearsSeropositive',
            y = 'exec_domain_z',
            ax=ys_ax)

fig.tight_layout()

### Q1: By inspection, which variable is most correlated?

|               |    |
| --------------|----|
| Points        | 5  |
| Public Checks | 3  |

_Points:_ 5

In [ ]:
# Answer: age, education, YearsSeropositive
q1_most_correlated = ...

In [ ]:
grader.check("q1_initial_correlation")

In [ ]:
fig, (race_ax, sex_ax, art_ax) = plt.subplots(1,3, sharey=True, figsize = (15, 5))

sns.stripplot(data=data,
            x = 'race',
            y = 'exec_domain_z', ax=race_ax)
sns.boxplot(data=data,
            x = 'race',
            y = 'exec_domain_z', ax=race_ax)

sns.stripplot(data=data,
            x = 'sex',
            y = 'exec_domain_z', ax=sex_ax)
sns.boxplot(data=data,
            x = 'sex',
            y = 'exec_domain_z', ax=sex_ax)

sns.stripplot(data=data,
            x = 'ART',
            y = 'exec_domain_z', ax=art_ax)
sns.boxplot(data=data,
            x = 'ART',
            y = 'exec_domain_z', ax=art_ax)

### Q2: By inspection, which variable has the most between class difference?

|               |    |
| --------------|----|
| Points        | 5  |
| Public Checks | 3  |

_Points:_ 5

In [ ]:
# Answer: race, sex, ART
q2_most_bcd = ...

In [ ]:
grader.check("q2_initial_bcd")

## Basic regression

We'll start by taking the simplest approach and regress the most correlated value first.

`pg.linear_regression` works by regressing all columns in the first parameter against the single column in the second.
By convention, we usually use the variables `X` and `y`.

You'll often see this written as:

$\mathbf{y} = \mathbf{X} \boldsymbol{\beta} + \boldsymbol{\epsilon}$

In the case of `pg.linear_regression` the $\boldsymbol{\epsilon}$ is added by default and we do not need to specify it.

You do not have to use the variable names `X` and `y`, in many cases you might have multiple `X`s and `y`s, but for simplicity, I will stick with this simple convention.

In [ ]:
X = data['YearsSeropositive'] # Our independent variables
y = data['exec_domain_z']     # Our dependent variable
res = pg.linear_regression(X, y)
res

This has fit the equation:

`PDZ = -0.035*YS + 0.712`

It tells us that the likelihood of this slope being zero is 1.0E-20 and that years-seropositive explains ~23.6% of variation in EDZ that we observe.

In [ ]:
ax = sns.regplot(data = data,
                 x = 'YearsSeropositive',
                 y = 'exec_domain_z')

# Pick "years seropositive" from 0 to 70
x = np.arange(0, 70)

# Use the coefficients from above in a linear equation
y = res.loc[1, 'coef']*x + res.loc[0, 'coef']

ax.plot(x, y, color = 'r')

## Residuals

_Residuals_ are the difference between the observed value and the predicted value.
In the case of a simple linear regression, this is the y-distance between each point and the best-fit line.
Examining these is an import step in assessing the fit for any biases.
You can think of the residual as what is "left over" after the regression.

We could calculate these ourselves from the regression coefficients, but, `pingouin` conviently provides them for us.
The result `DataFrame` from `pg.linear_regression` has a special attribute `.residuals_` which stores the difference between the prediction and reality for each point in the dataset.

In [ ]:
print(res.residuals_[:5])

In order to test the **Homoscedasticity** we want to ensure that these residuals are _not correlated with the depenendant variable_.

In our case, this means that the model is equally good predicting the EDZ of people recently infected with HIV and those who have been living with HIV for a long time.

To do this, we plot the residuals vs each independent variable.

In [ ]:
sns.scatterplot(x=data['YearsSeropositive'],  y=res.residuals_)

This is an ideal residual plot.
It should look like a random "stary-night sky" centered around 0.
This implies that the model is not better or worse for any given X value.

Let's also test our assumption about a normal distribution of errors of the residuals.

### Q3: Are the residuals normally distributed?

|               |    |
| --------------|----|
| Points        | 5  |
| Public Checks | 5  |

_Points:_ 5

In [ ]:
# Create a Q-Q plot of the residuals

q3_plot = ...

In [ ]:
# Use the Jarque-Bera normal test for large sample sizes

q3_norm_res = ...

In [ ]:
# Are the residuals normally distributed? 'yes' or 'no'

q3_is_norm = ...

In [ ]:
grader.check("q3_resid_normality")

You don't need to do this test at every stage, but it is a good test to do before you are _done_.

## Multiple Regression

Regression is not limited to a single independent variable, you can add as many as you'd like.

In our case, there are two others that we should consider: `age` and `education`

In [ ]:
X = data[['YearsSeropositive', 'education', 'age']]
y = data['exec_domain_z']
res = pg.linear_regression(X, y)
res

Now, it has fit the equation:

`EDZ = -0.037*YS - 0.103*edu + 0.019*age + 0.977`

The education is significant at p=8.17E-7.
Be caution when comparing coefficients, we might be tempted to compare -0.0422 and -0.0506 and say that education has a more negative effect than YS ...
But, remember that education ranges from 0-12 and YS ranges from 0-60, these are not on the same scale and are not directly comparable.
We'll talk about how to compare relative importance later.

As before, we should check the residuals of the model against _each_ independent variable in the regression to check for homoscedasticity.

In [ ]:
fig, (ys_ax, edu_ax, age_ax) = plt.subplots(1,3, sharey=True, figsize = (15, 5))

sns.scatterplot(x=data['YearsSeropositive'],  y=res.residuals_, ax=ys_ax)
sns.scatterplot(x=data['education'],  y=res.residuals_, ax=edu_ax)
sns.scatterplot(x=data['age'],  y=res.residuals_, ax=age_ax)

Three more stary night skies. Perfect.

Remember, the residual is the difference between the prediction of the model and reality.
Therefore, we can also use the residual plots to see how well the regression is handling other variables we have not included in the model.
If the model has properly accounted for something, the residual plot should stay centered around 0.

This can be done for categorical or continious variables.

In [ ]:
fig, (race_ax, sex_ax, art_ax) = plt.subplots(1,3, sharey=True, figsize = (15, 5))

race_ax.set_ylabel('residual')

sns.barplot(x=data['race'],  y=res.residuals_, ax=race_ax)
sns.barplot(x=data['sex'],  y=res.residuals_, ax=sex_ax)
sns.barplot(x=data['ART'],  y=res.residuals_, ax=art_ax)

Here we see some interesting patterns:
 - The graph of race against residuals shows us that our model is signifacntly racially biased. AA individuals are significantly 'under-estimated' by the model, C individauals are significantly over-estimated, and H individuals are significantly over-estimated.
 - The graph of sex shows that there is no real difference in the residuals. It has accounted for sex already.
 - It looks like there is a real difference across ART.

## _ANCOVA_

What we have done above is create a model that _accounts_ for the effects of age, education, and YS on EDZ.
We **subtracted** that effect (the predicted value) from the observed value thus creating the _residual_.
This is what is "left over" in the observed value after accounting for covariates or nuisance variables.
Then we plotted the _residual_ against each of our categorical variables.
If we then took the ANOVA of these residuals we'd be testing the hypothesis:
 _When accounting for age, education, and YS is there a difference across race._
 
This process is called an _Analysis of covariance_ or an **ANCOVA**.

### Standard first

### Q4: Perform an ANOVA between ART on the Executive Domain Z-score.

|               |    |
| --------------|----|
| Points        | 5  |
| Public Checks | 4  |

_Points:_ 5

In [ ]:
# Create a plot showing the effect of ART on EDZ
q4_plot = ...

In [ ]:
# Perform an ANOVA testing the impact of ART on EDZ
q4_res = ...
q4_res

In [ ]:
# Does ART have a significant impact on Executive Domain? 'yes' or 'no'?

q4_art_impact = ...

In [ ]:
grader.check("q4_art_test")

### With correction

Nicely `pingouin` has something built right in to do this whole process.

In [ ]:
sns.barplot(x=data['ART'],  y=res.residuals_)

# An ANCOVA testing the impact of ART on EDZ
# after correcting for the impace of age, education and YS
pg.ancova(data,
          dv = 'exec_domain_z',
          between = 'ART',
          covar=['YearsSeropositive', 'education', 'age'])

We can notice that after correction for covaraites the F-value has increased and the p-value has decreased.
This means the analysis is attributing more difference to race after correction and is more sure this is not due to noise.

The _advantage_ of using the `pg.ancova` function is that you can easily and quickly do your analysis.
The _disadvantage_ is that you cannot examine the internal regression for Normality and Homoscedasticity.

But, what if we wanted to have a covariate that is a category like race?

## Regression with categories

So, how do you do regression with a category like race?

Could it be as simple as adding it the `X` matrix?

In [ ]:
# X = data[['YearsSeropositive', 'education', 'age', 'race']]
# y = data['processing_domain_z']
# res = pg.linear_regression(X, y)
# res

Would have been nice, but we need to get a little tricky and use _dummy_ variables.

In their simplest terms, dummy variables are binary representations of categories.
Like so.

In [ ]:
pd.get_dummies(data['race']).head()

In [ ]:
# Extracting the same continious variables
X = data[['YearsSeropositive', 'education', 'age']]

# Creating new dummy variables for race
dummy_vals = pd.get_dummies(data['race']).astype(float)


# Adding them the end
X = pd.concat([X, dummy_vals], axis=1)

y = data['exec_domain_z']

res = pg.linear_regression(X, y)
res.round(3)

This _Warning_ is telling us that our model has fallen into the _dummy variable trap_.
The dummy variable trap occurs when dummy variables created for categorical data in a regression model are perfectly collinear, meaning one variable can be predicted from the others, leading to redundancy.
This happens because the inclusion of all dummy variables for a category along with a constant term (intercept) creates a situation where the sum of the dummy variables plus the intercept equals one, introducing perfect multicollinearity.
To avoid this, one dummy variable should be dropped to serve as the reference category, ensuring the model's design matrix is full rank and the regression coefficients are estimable and interpretable.

In [ ]:
pd.get_dummies(data['race'], drop_first=True).head()

In [ ]:
X = data[['YearsSeropositive', 'education', 'age']]
dummy_vals = pd.get_dummies(data['race'], drop_first=True).astype(float)
X = pd.concat([X, dummy_vals], axis=1)
y = data['exec_domain_z']
res = pg.linear_regression(X, y)
res.round(3)

We can notice a few things here:
 - **AA** has become the 'reference', the coefficients of C and H are relative to AA, which is set at 0.
   - C individuals have a decreased score (relative to AA), which is significant.
   - H individuals have an decreased score (relative to AA), which is significant.

We can look at the residuals.

In [ ]:
fig, (race_ax, sex_ax, art_ax) = plt.subplots(1,3, sharey=True, figsize = (15, 5))

race_ax.set_ylabel('residual')

sns.barplot(x=data['race'],  y=res.residuals_, ax=race_ax)
sns.barplot(x=data['sex'],  y=res.residuals_, ax=sex_ax)
sns.barplot(x=data['ART'],  y=res.residuals_, ax=art_ax)

Let's merge everything into a single analysis.

In [ ]:
X = pd.concat([data[['YearsSeropositive', 'education', 'age']],
               pd.get_dummies(data['race'], drop_first=True).astype(float),
               pd.get_dummies(data['sex'], drop_first=True).astype(float),
               pd.get_dummies(data['ART'], drop_first=True).astype(float),
              ], axis=1)
y = data['exec_domain_z']
res = pg.linear_regression(X, y)
res.round(3)

Here our _reference_ is an AA, female taking Stavudine.
 - Everything is signifiant except for sex.
 - We see that Truvada has a _significant positive_ effect on EDZ relative to Stavudine.

Since this is our final model, let's test our last normality assumption.

In [ ]:
pg.qqplot(res.residuals_)

In [ ]:
pg.normality(res.residuals_, method='normaltest')

Perfect, now we know that our final model passes the _Normal Distribution of Errors_ assumption.

What about understanding which parameters have the largest impact on the model?
Stated another way: which features are most important to determing EDZ?

Nicely, `pingouin` can do this for us.

In [ ]:
res_with_imp = pg.linear_regression(X, y, relimp=True)
res_with_imp

In [ ]:
# After filtering and sorting
res_with_imp.query('pval<0.01').sort_values('relimp_perc', ascending=False)

## Over fitting

In principle we can continue to add more and more variables to the `X` and just let the computer figure out the p-value of each.

There are a few reasons we shouldn't take this tack.
 - **Overfitting** : A larger model will **ALWAYS** fit better than a smaller model. This doesn't mean the larger model is **better** at predicting _all samples_, it just means it fits **these** samples better.
 - **Explainability** : Large models with many parameters are difficult to explain and reason about. We are biologists, not data scientists. Our job is to reason about the _result_ of the analysis, not create the best fitting model.
 - **Statistical power** : As you add more noise features you lose the power to detect real features.

So, you should limit yourself to only those features that you think are biologically meaningful.

When planning experiments there are a couple of things you can do to avoid overfitting:
 - **Sample size** : While there is no strict rule, you should plan to have _at least_ 10 samples per feature in your model.
 - **Even sampling** : It is ideal to have a roughly equal representation of the entire parameter space. If you have categories, you should have an equal number of each. If you have continious data, you should have both high and low values. If you have many parameters, you should have an equal number of each of their interactions as well.

These are good guidelines for all model-fitting style analyses.

In [ ]:
print('Features:', len(X.columns))
print('Obs:', len(X.index))

## Even more regression

There are a number of regression based tools in `pingouin` that we didn't cover that may be useful to explore.
 - `pg.logistic_regression` : This works similar to linear regression but is for binary dependent variables.
Each feature is regressed to create an equation that estimates the likelihood of the `dv` being `True`.
 - `pg.partial_corr` : Like the ANCOVA, this is a tool for removing the effect of covariates and then calculating a correlation coefficient.
 - `pg.rm_corr` : Correlation with repeated measures. This is useful if you have measured the same _sample_ multiple times and want to account for intermeasurment variability.
 - `pg.mediation_analysis` : Tests the hypothesis that the independent variable `X` influences the dependent variable `Y` by a change in mediator `M`; like so `X -> M -> Y`.
This is useful to disentangle causal effects from covariation.

---------------------------------------------

## Submission

You do not need to submit this walkthrough notebook.
Simply complete the quiz.

In [ ]:
grader.check_all()